# SPECTRALISM

An introduction the Spectrum class and musx algorithms for composing with spectral data.

<hr style="height:1px;color:gray">

Notebook imports:

In [1]:
import sys
sys.path.append('/Users/taube/Software/musx')
from IPython.display import display, HTML, Audio
display(HTML("<style>.container { width:100% !important; }</style>"))
#from IPython IPython.display.Audio("my_audio_file.mp3")
import math
from musx import Score, Note, MidiEvent, Seq, MidiFile, Cycle, Choose, Shuffle, version, \
setmidiplayer, playfile, import_spear_frames, PCSet, rmspectrum, fmspectrum, hertz, pitch, \
keynum, scale, intempo, odds, pick, between
from musx.midi.gm import Koto, Flute, Marimba, Clarinet
print(f'version: {version}')

version: N.N.N


This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

## The Spectrum class

<!-- In spectralism, synthesis and audio analysis algorithms are tools for composing music based on acoustic information. A *spectral composer* adopts both  This notebook shows how this can be done using two different audio algorithms. -->

The musx Spectrum class is contains frequency and amplitude pairs with methods that operate on this information for  compositional purposes.  Spectrums can be created by loading spectral datafiles created from
Michael Klingbeil's remarkable <a href="https://www.klingbeil.com/spear/">SPEAR</a> application, or directly computed by the `fmspectum()` and `rmspectum()` functions in musx. Given a spectrum the composer can convert its data into compositional forms such as note sets, pitch classes, amplitude envelopes, etc.

### Spectral information


There are two basic approaches to spectral composition:

- Recorded sound is analyzed for its spectral content using software that provides FFT and sonogram sevices. This spectral information is then used as source material that composers manipulate, orchestrate, convert into note sets, melodic tropes, harmonic progressions etc.
- Frequency (notes) are directly *synthesized* using audio algorithms, then organized by the composer and performed by instrumentalists.

In this example, spectral information from a single log-drum note is analyzed in Spear and imported into python as musx Spectrum objects, then analyzed for its characteristics and used as note data in a composition.

[Click here](support/log-drum.txt) to see the raw spectral data exported from SPEAR. 

Use this audio control to listen to the sound:

<audio controls=true src="support/log-drum.mp3"/>

The first step in the process is to import Spear's spectral frame data into Python using `import_spear_frames()`.  Each frame will be represented in Python as a `Spectrum` object. A Spectrum is a sorted list of pairs: [[*freq1*, *amp1*], [*freq2*, *amp2*], ... [*freqN*, *ampN*]] and when it is printed it displays the number of pairs it contains:

In [ ]:
frames = import_spear_frames('./support/log-drum.txt')
print(f"Imported {len(frames)} frames:\n{frames}")

Take the sixth spectrum to use as an example:

In [ ]:
spec = frames[5]
print(spec)

Spectrum pairs are sorted by frequency:

In [ ]:
print(spec.pairs())

Access the spectrum's frequencies:

In [ ]:
print(spec.freqs())

Access it's amplitudes:

In [ ]:
print(spec.amps())

Print the minimum and maximum frequencies:

In [ ]:
print(f"min freq: {spec.minfreq()}, max freq: {spec.maxfreq()}")

Print the minimum and maximum amplitudes:

In [ ]:
print(f"min amp: {spec.minamp()}, max amp: {spec.maxamp()}")

The `keynums(quant=None, unique=None, minkey=0, maxkey=127, thresh=0)` function converts hertz frequencies into floating point key numbers *kkk.nnn*, where the fractional values *nnn* are cents above *kkk*.  The method is really a "swiss army knife" for converting spectral frequencies into easily understood compositional information. Here are some of the things it can do:` 

In [ ]:
print(spec.keynums())

Return frequencies quantized to integer key numbers:

In [ ]:
print(spec.keynums(quant=1))

Return frequencies quantized to quarter tones:

In [ ]:
print(spec.keynums(quant=.5))

Return frequencies quantized to integer whole steps:

In [ ]:
print(spec.keynums(quant=2))

The quant value can also be a function:

In [ ]:
print(spec.keynums(quant=math.floor))

The `minkey` and `maxkey` parameters force a returning keynum to lie above a specified minimum and/or below a maximum value:

In [ ]:
print(spec.keynums(quant=1, minkey=60, maxkey=80))

If `unique` is True then any duplicate key numbers will be removed:

In [ ]:
print(spec.keynums(quant=1, unique=True, minkey=60, maxkey=71))

This example computes a pitch class set and matrix from the timbre of a log drum recording =:)

In [ ]:
pcs = [k % 12 for k in spec.keynums(quant=1, unique=True, minkey=60, maxkey=71)]
print(f"pcs: {pcs}\nset: {PCSet(pcs)}\nmatrix:")
PCSet(pcs).matrix().print()

### Musical example

Once a spectrum has been converted to key numbers, pitch classes, or Pitches, it is in a format that any composer can imagine how music could be created from it. 

This simple example treats each Spectrum (log drum frame) as a chord to strum and sing.

In [ ]:
def strum_spectra (score, specs, dur, rhy):
    strum = .1
    for s in specs:
        for k in s.keynums():
            m = Note(time=score.now, duration=dur, pitch=k, 
                     amplitude=.7, instrument=0)
            score.add(m)
            yield strum
        yield rhy - (strum * s.size())
        
def sing_spectra (score, specs, dur, rhy):
    for s in specs:
        for k in s.keynums():
            m = Note(time=score.now, duration=rhy+.5, pitch=k, 
                     amplitude=.20, instrument=2)
            score.add(m)
        yield rhy
        
meta = MidiFile.metatrack(ins={0: Koto, 2: 52}, microdivs=2)
score = Score(out=Seq())
score.compose([strum_spectra(score, frames, 2, 6), 
               sing_spectra(score, frames, 2, 6)])

file = MidiFile("spectralism.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Spectral syntheses

### Ring Modulation

In ring modulation two signals f1 and f2 are multiplied, which results in *sum and difference* tones: f1+f2 and |f1-f2|. 

<img src="support/rm1.png" width=300 />

If f1 and f2 are sine waves then the output spectrum consists of two sidebands, and if either f1 or f2 is a complex wave the output spectrum consists of the sum and difference tones for each harmonic in both signals.

This results in an output signal with the following characteristics:

* The output spectrum is not related to the inputs by a harmonic relationship because partials are additive not multiplicative.

* f1 and f2 are not present in the resulting spectrum.

* The fact that each partial in f1 produces the sum and difference with every partial in f2 can result in a very dense output spectrum.

----
`rmspectrum(reqs1, freqs2, asfreqs=False)`

The musx function `rmspectrum()` returns the ring-modulated spectrum of two input signals, either of which can be a frequency, a list of frequencies, or a Spectrum of frequencies.  The `asfreqs` parameter allows the output to be returned as either a Spectrum or a list of frequencies.

The next example performs a series of spectra immediately followed by their ring modulated versions. In each input string the first note (C5) becomes first input spectrum and the remaining notes constitute the second input spectrum, resulting in the following set of frequencies.

The first half notes in each measure are the input spectra and the second are their ring modulated versions:
<img src="support/rm2.png" width=550 />

In [ ]:
def rmchords(score, sets, dur):
    for set in sets:
        res = rmspectrum(set[0], set[1:])
        for k in set:
            m = Note(time=score.now, pitch=keynum(k), duration=dur)
            score.add(m)
        # print(pitch(set, hz=True), " -> ", pitch(res.keynums()))
        for k in res.keynums():
            m = Note(time=score.now + dur, pitch=k, duration=dur)
            score.add(m)
        yield dur*2

inputs = hertz(["C5 b", "C5 bf", "C5 a", "C5 af", "C5 g",
                "C5 fs", "C5 f", "C5 e", "C5 ef", "C5 d", "C5 db",
                "C5 ef b", "C5 f bf", "C5 f b", "C5 fs b",
                "C5 af b", "C5 f fs b", "C5 d f g a"])

meta = MidiFile.metatrack(ins= {0: Flute})
score = Score(out=Seq())
score.compose(rmchords(score, inputs, .75))
file = MidiFile("spectralism.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

### Example: ring modulation etude

This etude performs the input and output spectra from ring modulation using a melody and accompaniment texture sounding a bit like a hurdy-gurdy circus ensemble. The melodic part (clarinet) performs the modulated output spectrum and the accompaniment (marimba) plays the input spectrum. The `main()` part composer sprouts the melody and accompaniment part composers passing them randomized input and output spectra:

In [ ]:
def melody(score, reps, dur, set3):
    # Create a cycle of the output set.
    pat = Cycle(set3)
    for _ in range(2 * reps):
        m = Note(time=score.now, duration=dur/2, pitch=pat.next(), amplitude=.7, instrument=1)
        score.add(m)
        # Wait till the next note
        yield dur
        
def accompaniment(score, reps, dur, set1, set2):
    # Create a cycle of the two inputs
    pat = Cycle([set1, set2])
    for _ in range(reps*2):
        # Get the next set.
        keys = pat.next()
        # Iterate the keys, play each as a chord.
        for k in keys:
            # Create a midi note at the current time.
            m = Note(time=score.now, duration=dur, pitch=k, amplitude=.3, instrument=0)
            # Add it to our output seq.
            score.add(m)
        # Wait till the next chord.
        yield dur

def main(score, reps, dur, keys):
    # sprout the melody and accompanyment for each measure 
    num = Choose([1,2,3])
    # scramble the cycle of fourths
    pat = Shuffle(keys)
    for _ in range(reps):
        # input1 is 1, 2 or 3 notes from cycle of 4ths
        keys1 = [pat.next() for _ in range(num.next())]
        # input2 is same
        keys2 = [pat.next() for _ in range(num.next())]
        # ring modulate the inputs
        spect = rmspectrum([hertz(k) for k in keys1], [hertz(k) for k in keys2])
        # convert output spectrum to keynums
        keys3 = spect.keynums(quant=1, unique=True, minkey=21, maxkey=108)
        # sprout composers to play inputs and outputs
        playn = pick(3,4,5)
        score.compose(accompaniment(score, playn, dur, keys1, keys2))
        #shuffle(keys3)
        #print(keys2)
        score.compose(melody(score, playn, dur, Shuffle(keys3).next(True)))
        # do it again after composers finish
        yield (dur * playn * 2)
        
meta = MidiFile.metatrack(ins={0: Marimba, 1: Clarinet})
score = Score(out=Seq())
keys = scale(40, 12, 5)
rhy = intempo(.25, 74)
score.compose( main(score, 24, rhy, keys) )
file = MidiFile("spectralism.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Frequency Modulation

In Frequency Modulation (FM), the frequency of a carrier wave is altered by a modulator signal:

<img src="support/fm1.png" width=200/>   <img src="support/fm2.png" width=200 />

Deviations of the carrier (c) cause spectral sidebands to appear at multiples of the modulator (m) above and below the carrier:

c ± (k*m)  where k=sideband order  0,1,2...

<img src="support/fm3.png" width=350 />

### The Carrier to Modulator Ratio

* The carrier to modulator ratio (C:M) determines the harmonicity of the resulting spectrum.

* For harmonic spectra, M must have an integer relationship to C.

* Non-integer ratios produce inharmonic spectra.

<img src="support/fm4.png" width=400 />

### The FM Index

* The FM index (I) controls the amount of frequency deviation around the carrier: dev=m* I

* The higher the index the more sidebands are active: k=round(I)+1;  spectral density=2k+1

<img src="support/fm5.png" width=800 />

### Composing with FM

FM is a powerful algorithm for generating spectral note sets!

* It can generate inharmonic (dissonant) or harmonic (consonant) sets.

* It requires only three basic parameters:
    * carrier (center frequency)
    * c/m ratio (carrier/modulator ratio)
    * index (density, or width, of the spectrum.)

`fmspectrum(carrier, ratio, index)`

The musx `fmspectrum()` function returns an FM generated Spectrum given a carrier (in hertz), a carrier to modulator ratio, and an FM index.  This example produces a pure harmonic series based on 100 Hz:

In [ ]:
fmspec = fmspectrum(400, 1/4, 8)
print(fmspec.freqs())

This example produces an inharmonic spectrum based on 100 Hz:

In [ ]:
fmspec = fmspectrum(100, math.e, 3)
print(fmspec.freqs())

Converting the hertz values into floating point key numbers, notes and pitch classes gives us a better idea of the possible compositional materials produced by this spectrum:

In [ ]:
print(f"keynums: {fmspec.keynums()}")

print(f"\npitches: {pitch(fmspec.keynums(minkey=36, maxkey=84, unique=True))}")

print(f"\npcset: {PCSet([round(k) % 12 for k in fmspec.keynums(minkey=36, maxkey=84, unique=True)])}")

### Example: Fluctuating Harmony

Listen to a rising series of microtonal FM generated chords with random fluctuations in their c:m ratio and index:

In [ ]:
def fmchords(score, center, cm1, cm2, in1, in2, rhy, tune):
    for c in center:
        carrier = hertz(c)
        cmrat = between(cm1, cm2)
        index = between(in1, in2)
        spec = fmspectrum(carrier, cmrat, index)
        #print("spectrum freqs:", spec.freqs())
        keys = spec.keynums(minkey=c-12, maxkey=c+12)
        #print("spectrum keynums:", keynums)
        for k in keys:
            note = Note(time=score.now, pitch=k, duration=rhy)
            score.add(note)
        yield .75

meta = MidiFile.metatrack(microdivs=1)
score = Score(out=Seq())
centers = [i for i in range(50, 71, 2)]
score.compose(fmchords(score, centers, 1.0, 2.0, 2.0, 3.0, .75, 4))
file = MidiFile("spectralism.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

### Example: FM improvisor

This example is an improvisor where FM spectra produce melodic gestures 70% of the time and chordal structures 30% of the time. The improvisation is controlled by a carrier frequency contour line that moves around in a semi melodic manner.

In [ ]:
contour = keynum("a4 g f e a4 b c d gs b c5 ef fs gs "
                 "a5 bf g f e a5 b c d gs3 f e cs c " 
                 "bf5 gs5 as3 cs5 e6 f4 gs5 d6 e f g "
                 "c5 b a g bf c5 cs e4 f gs d4 c b "
                 "a4 e5 f g a5")

def fmimprov(score, line, beat):
    amp = .7
    dur = beat
    for knum in line:
        ismel = odds(.7)
        rhy = pick(dur, dur / 2, dur / 4)
        label = "melody" if ismel else "chord"
        f, r, x = hertz(knum), between(1.1, 1.9), pick(1, 2, 3)
        print(f"{label} -> carrier: {f}, c/m ratio: {r}, fm index: {x}")
        spec = fmspectrum(f,r,x)
        keys = spec.keynums(unique=True, minkey=knum-14, maxkey=knum+14)
        if ismel:
            keys = Shuffle(keys).next(True)
        sub = rhy / len(keys) if ismel else 0
        #print("melody:" if ismel else "chord:", "time=", q.now, "dur=", rhy, "keys=", keys)
        for i, k in enumerate(keys):
            m = Note(time=score.now + (i * sub), duration=dur, pitch=k, amplitude=amp)
            score.add(m)
        yield rhy     

meta = MidiFile.metatrack(microdivs=1) 
score = Score(out=Seq())
score.compose(fmimprov(score, contour, 1))
file = MidiFile("spectralism.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)